<a href="https://colab.research.google.com/github/haizzzi/lecture_notes/blob/main/preprocessing_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

regex에 관련된 실습

In [3]:
!head -n 5 /content/drive/MyDrive/nmt_fold/preproc/regex/review.sorted.uniq.tsv

negative	!
negative	! 다녹아서 왓어요 .. 짜증
negative	!!!!!!!!!!!!!
negative	!!!!이게뭐야?진짜이게뭐야?리뷰보고구매했는데!나원참!
negative	"싼 게 비지떡".200ml라고 판매하는데 내가 보기엔 절대 이거 100ml도 안된다.뚜껑 열다가 손톱 부러지고,향기는 에프킬라..구매후기 잘 안쓰는데 이건 진짜 최악이다.


In [4]:
!python /content/drive/MyDrive/nmt_fold/preproc/regex/refine.py /content/drive/MyDrive/nmt_fold/preproc/regex/refine.regex.txt 1 < /content/drive/MyDrive/nmt_fold/preproc/regex/review.sorted.uniq.tsv > /content/drive/MyDrive/nmt_fold/preproc/regex/review.sorted.uniq.refined.tsv

In [5]:
!head -n 5 /content/drive/MyDrive/nmt_fold/preproc/regex/review.sorted.uniq.refined.tsv

negative	!
negative	! 다녹아서 왓어요 .. 짜증
negative	!!!!!!!!!!!!!
negative	!!!!이게뭐야?진짜이게뭐야?리뷰보고구매했는데!나원참!
negative	"싼 게 비지떡".200ml라고 판매하는데 내가 보기엔 절대 이거 100ml도 안된다.뚜껑 열다가 손톱 부러지고,향기는 에프킬라..구매후기 잘 안쓰는데 이건 진짜 최악이다.


tokenization 실습 mecab 활용

mecab 설치

In [ ]:
# 1) 필수 도구
!apt-get -y update
!apt-get -y install curl

# 2) konlpy 공식 스크립트로 mecab + mecab-ko-dic 설치
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# 3) mecabrc의 dicdir 설정(로그에 나온 경로로 맞추기)
!sed -i 's|^dicdir =.*|dicdir = /usr/local/lib/mecab/dic/mecab-ko-dic|' /usr/local/etc/mecabrc

# 4) 파이썬 라이브러리
!pip install konlpy

tokenize.sh을 직접 실행시킴
---------------------------------


In [10]:
#tokenize.sh의 내용
#FN=$1
#TARGET_FN=$2

#cut -f1 ${FN} > ${FN}.label
#cut -f2 ${FN} | mecab -O wakati -b 20000 > ${FN}.text.tok

#paste ${FN}.label ${FN}.text.tok > ${TARGET_FN}

SyntaxError: invalid syntax (ipython-input-3063421299.py, line 5)

In [9]:
!bash /content/drive/MyDrive/nmt_fold/preproc/tokenization/tokenize.sh /content/drive/MyDrive/nmt_fold/preproc/tokenization/review.sorted.uniq.refined.tsv /content/drive/MyDrive/nmt_fold/preproc/tokenization/review.sorted.uniq.refined.tok.tsv

In [11]:
!head -n 5 /content/drive/MyDrive/nmt_fold/preproc/tokenization/review.sorted.uniq.refined.tok.tsv

negative	! 
negative	! 다 녹 아서 왓 어요 . . 짜증 
negative	!!!!!!!!!!!!! 
negative	!!!! 이게 뭐 야 ? 진짜 이게 뭐 야 ? 리뷰 보고 구매 했 는데 ! 나원참 ! 
negative	" 싼 게 비지떡 ". 200 ml 라고 판매 하 는데 내 가 보 기 엔 절대 이거 100 ml 도 안 된다 . 뚜껑 열 다가 손톱 부러지 고 , 향기 는 에프 킬 라 . . 구매 후기 잘 안 쓰 는데 이건 진짜 최악 이 다 . 


subword segmentation 관련 내용

In [ ]:
#tokenize.sh의 내용인데, 디렉토리는 코랩에 맞춰서 바꿈. 아래는 원래 버전

#!/bin/bash
#set -euo pipefail

#BASE="/content/drive/MyDrive/nmt_fold/preproc/subword_segmentation"
#cd "$BASE"

#export LANG=C.UTF-8
#export LC_ALL=C.UTF-8
#export PYTHONIOENCODING=utf-8

#FN="$1"              # review.sorted.uniq.refined.tsv
#TARGET_FN="$2"       # review.sorted.uniq.refined.tok.bpe.tsv
#N_SYMBOLS=30000

# 1) 레이블/텍스트 분리
#cut -f1 "${FN}" > "${FN}.label"
#cut -f2 "${FN}" > "${FN}.text"

# 2) 토크나이즈 (MeCab 버퍼 확대)
#cat "${FN}.text" \
#| mecab -b 10000000 -O wakati \
#| python post_tokenize.py "${FN}.text" \
#> "${FN}.text.tok"

# 3) BPE 입력 파일 UTF-8 보정 (필요시만 재인코딩)
#python - "$BASE/${FN}.text.tok" << 'PY'
#import sys, pathlib
#p = pathlib.Path(sys.argv[1])
#raw = p.read_bytes()
#for enc in ('utf-8','utf-8-sig','cp949','euc-kr'):
#    try:
#        raw.decode(enc)
#        if enc!='utf-8':
#            p.write_text(raw.decode(enc), encoding='utf-8')
#        break
#    except UnicodeDecodeError:
#        continue
#else:
#    p.write_text(raw.decode('utf-8', errors='ignore'), encoding='utf-8')
#PY

# 4) BPE 학습/적용
#python ./subword-nmt/learn_bpe.py --input "${FN}.text.tok" --output ./model --symbols "${N_SYMBOLS}"
#python ./subword-nmt/apply_bpe.py --codes ./model < "${FN}.text.tok" > "${FN}.text.tok.bpe"

# 5) 라벨과 붙이기
#paste "${FN}.label" "${FN}.text.tok.bpe" > "${TARGET_FN}"

In [17]:
!bash /content/drive/MyDrive/nmt_fold/preproc/subword_segmentation/tokenize.sh review.sorted.uniq.refined.tsv review.sorted.uniq.refined.tok.bpe.tsv

In [18]:
!head -n 5 /content/drive/MyDrive/nmt_fold/preproc/subword_segmentation/review.sorted.uniq.refined.tok.bpe.tsv

negative	▁▁!
negative	▁▁! ▁▁다 ▁녹 ▁아서 ▁▁왓 ▁어요 ▁▁. ▁. ▁▁짜증
negative	▁▁!!!!!!!!!!!!!
negative	▁▁!!!! ▁이게 ▁뭐 ▁야 ▁? ▁진짜 ▁이게 ▁뭐 ▁야 ▁? ▁리뷰 ▁보고 ▁구매 ▁했 ▁는데 ▁! ▁나원참 ▁!
negative	▁▁" ▁싼 ▁▁게 ▁▁비지떡 ▁" . ▁200 ▁ml ▁라고 ▁▁판매 ▁하 ▁는데 ▁▁내 ▁가 ▁▁보 ▁기 ▁엔 ▁▁절대 ▁▁이거 ▁▁100 ▁ml ▁도 ▁▁안 ▁된다 ▁. ▁뚜껑 ▁▁열 ▁다가 ▁▁손톱 ▁▁부러지 ▁고 ▁, ▁향기 ▁는 ▁▁에프 ▁킬 ▁라 ▁. ▁. ▁구매 ▁후기 ▁▁잘 ▁▁안 ▁쓰 ▁는데 ▁▁이건 ▁▁진짜 ▁▁최악 ▁이 ▁다 ▁.


detokenization 코드

In [19]:
!head -n 5 /content/drive/MyDrive/nmt_fold/preproc/detokenization/review.sorted.uniq.refined.tok.bpe.tsv

negative	▁▁!
negative	▁▁! ▁▁다 ▁녹 ▁아서 ▁▁왓 ▁어요 ▁▁. ▁. ▁▁짜증
negative	▁▁!!!!!!!!!!!!!
negative	▁▁!!!! ▁이게 ▁뭐 ▁야 ▁? ▁진짜 ▁이게 ▁뭐 ▁야 ▁? ▁리뷰 ▁보고 ▁구매 ▁했 ▁는데 ▁! ▁나원참 ▁!
negative	▁▁" ▁싼 ▁▁게 ▁▁비지떡 ▁" . ▁200 ▁ml ▁라고 ▁▁판매 ▁하 ▁는데 ▁▁내 ▁가 ▁▁보 ▁기 ▁엔 ▁▁절대 ▁▁이거 ▁▁100 ▁ml ▁도 ▁▁안 ▁된다 ▁. ▁뚜껑 ▁▁열 ▁다가 ▁▁손톱 ▁▁부러지 ▁고 ▁, ▁향기 ▁는 ▁▁에프 ▁킬 ▁라 ▁. ▁. ▁구매 ▁후기 ▁▁잘 ▁▁안 ▁쓰 ▁는데 ▁▁이건 ▁▁진짜 ▁▁최악 ▁이 ▁다 ▁.


In [20]:
!python /content/drive/MyDrive/nmt_fold/preproc/detokenization/detokenizer.py < /content/drive/MyDrive/nmt_fold/preproc/detokenization/review.sorted.uniq.refined.tok.bpe.tsv > /content/drive/MyDrive/nmt_fold/preproc/detokenization/review.sorted.uniq.refined.tok.bpe.detok.tsv

In [21]:
!head -n 5 /content/drive/MyDrive/nmt_fold/preproc/detokenization/review.sorted.uniq.refined.tok.bpe.detok.tsv

negative	!
negative	! 다녹아서 왓어요 .. 짜증
negative	!!!!!!!!!!!!!
negative	!!!!이게뭐야?진짜이게뭐야?리뷰보고구매했는데!나원참!
negative	"싼 게 비지떡".200ml라고 판매하는데 내가 보기엔 절대 이거 100ml도 안된다.뚜껑 열다가 손톱 부러지고,향기는 에프킬라..구매후기 잘 안쓰는데 이건 진짜 최악이다.
